In [25]:
import os

In [26]:
%pwd

'/home/vinayak.t'

In [27]:
os.chdir("../")


In [28]:
%pwd

'/home'

In [29]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [30]:
from IgfCnnClassifier.constants import *   
from IgfCnnClassifier.utils.common import read_yaml, create_directories


ModuleNotFoundError: No module named 'IgfCnnClassifier'